<a href="https://colab.research.google.com/github/Mofek/Mofek/blob/main/R21_13_october_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata


In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
!pip install json-repair
!pip install --upgrade --quiet  langchain-openai langchain-community
!pip install  py2neo
!pip install PyPDF2
!pip install neo4j
!pip install --upgrade --quiet  langchain-openai langchain-community
!pip install pandas
!pip install langchain
!pip install pydantic
!pip install faiss-gpu




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.

In [4]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
import pandas as pd
from py2neo import Graph, Node, Relationship
import PyPDF2
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader


from langchain_core.documents import Document

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from google.colab import output
output.enable_custom_widget_manager()
import yaml

from dotenv import load_dotenv

load_dotenv()


True

# **Load Data section**

In [ ]:
#if you want to load pdf file use this code
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

pdf_path = '/content/2023-Texas-Cancer-Registry-Annual-Report.pdf'  # Replace with your actual PDF path
text = extract_text_from_pdf(pdf_path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents1 = text_splitter.split_documents([Document(page_content=text)])
print(documents1[:10])

[Document(metadata={}, page_content='2023 Texas Cancer Regist ry  \n                         Annual Report\n \n \nAs Required by  \n       Texas Health and Safety Code,  \n    Section 82.007  \n \n \nSeptember  2023 \n i \n Table of Contents'), Document(metadata={}, page_content='Table of Contents  \nExecutive Summary  ................................ ................................ ...............  2 \nIntroduction  ................................ ................................ ...........................  3'), Document(metadata={}, page_content='1. Cancer in Tex as ................................ ................................ .................  5 \nCancer Incidence  ................................ ................................ ................  5'), Document(metadata={}, page_content='Cancer Mortality  ................................ ................................ .................  6 \nCancer Survival  ................................ ................................ 

In [ ]:
#USE FOR CSV FILE
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Step 1: Load the CSV file
csv_file_path = '/content/HumanOmniExpress-12-v1-0-K.csv'
df = pd.read_csv(csv_file_path, sep=';',on_bad_lines='skip', engine='python')

# Step 2: Concatenate the text data
# Assuming the text you want to process is in a column named 'text'
# Modify 'text' to the appropriate column name in your CSV
column_name = df.columns[0]
text_data = " ".join(df[column_name].astype(str).tolist())

# Create a Document object from the text data
docs = [Document(page_content=text_data)]

# Step 3: Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
documents = documents[:100]
# Check the result
for i, doc in enumerate(documents[:10]):
    print(f"Chunk {i+1}: {doc.page_content}\n")
documents.extend(documents1)
print(documents[:1000])



In [ ]:
#if you want to load TXT file use this code

loader = TextLoader(file_path="/content/dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
print(documents[:10])

In [ ]:
llm_type = os.getenv("LLM_TYPE", "GPT_MODEL")
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [ ]:
llm_type = os.getenv("LLM_TYPE", "GPT_MODEL")
llm = ChatOpenAI(temperature=0, model="gpt-4o")

#template = """Please help me generate a knowledge graph from the following data and descriptions """

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents1)


In [ ]:
graph_documents[0]

GraphDocument(nodes=[Node(id='American Lung Association', type='Organization', properties={}), Node(id='State Of Lung Cancer Report', type='Report', properties={}), Node(id='Lung Cancer', type='Disease', properties={}), Node(id='State', type='Location', properties={})], relationships=[Relationship(source=Node(id='American Lung Association', type='Organization', properties={}), target=Node(id='State Of Lung Cancer Report', type='Report', properties={}), type='PUBLISH', properties={}), Relationship(source=Node(id='State Of Lung Cancer Report', type='Report', properties={}), target=Node(id='Lung Cancer', type='Disease', properties={}), type='ANALYZE', properties={}), Relationship(source=Node(id='State Of Lung Cancer Report', type='Report', properties={}), target=Node(id='State', type='Location', properties={}), type='COMPARE', properties={})], source=Document(metadata={}, page_content='How does your state compare?\ue913 SELECT LOCATION\nThe American Lung Association\'s "State of Lung Canc

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

# ***Graph presentation***

In [ ]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

In [ ]:
from neo4j import GraphDatabase

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    uri=os.environ["NEO4J_URI"],
    auth=("neo4j", os.environ["NEO4J_PASSWORD"])
)

# Function to get all node labels
def get_all_node_labels(driver):
    with driver.session() as session:
        result = session.run("""
            MATCH (n)
            UNWIND labels(n) AS label
            RETURN DISTINCT label
        """)
        node_labels = [record["label"] for record in result]
    return node_labels

# Get all node labels
node_labels = get_all_node_labels(driver)
print("Node Labels:", node_labels)


Node Labels: ['sample', 'Sex', 'county', 'Population', 'Superpopulation', 'Data_collections', 'state', 'Cancer_Type', 'Chromosome', 'SNP', 'Gene', 'Allele']


In [ ]:
def get_string_property_keys(driver):
    with driver.session() as session:
        result = session.run("""
            MATCH (n)
            UNWIND keys(n) AS key
            WITH DISTINCT key
            RETURN key
        """)
        property_keys = []
        for record in result:
            key = record["key"]
            # Check if the property values are strings
            prop_result = session.run(f"""
                MATCH (n)
                WHERE n.`{key}` IS NOT NULL
                RETURN n.`{key}` AS value LIMIT 1
            """)
            value = prop_result.single()["value"]
            if isinstance(value, str):
                property_keys.append(key)
        return property_keys

# Get string property keys
property_keys = get_string_property_keys(driver)
print(property_keys)

['Sample_name', 'Sex', 'County', 'Population_code', 'Population_name', 'Superpopulation_name', 'Superpopulation_code', 'Data_collections', 'State', 'Cancer_Type', 'Gene', 'alt_allele', 'id', 'Gene_symbol', 'ref_allele', 'Gene_Symbol', 'Gene_name', 'name', 'nucleotide']


In [ ]:
node_label_string = node_labels[0] if node_labels else None

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model='text-embedding-3-large',dimensions=384),
    search_type="hybrid",
    node_label="sample",
    text_node_properties=property_keys,
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()


/usr/local/lib/python3.10/dist-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_community.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
text_embeddings = embeddings.embed_documents(texts)
text_embedding_pairs = list(zip(texts, text_embeddings))
vectorstore = Neo4jVector.from_embeddings(
    text_embedding_pairs, embeddings)


<ipython-input-11-2bd86500b133>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


NameError: name 'texts' is not defined

# **Structered Data**

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
!pip install json-repair
!pip install  py2neo
!pip install neo4j
!pip install --upgrade py2neo



In [ ]:
import pandas as pd
import os

from py2neo import Graph, Node, Relationship
#from neo4j_runway import Discovery, GraphDataModeler, PyIngest, UserInput
#from neo4j_runway.code_generation import PyIngestConfigGenerator
#from neo4j_runway.llm.openai import OpenAIDiscoveryLLM, OpenAIDataModelingLLM
from dotenv import load_dotenv
import numpy as np
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader

#from neo4j_runway import Discovery, GraphDataModeler, IngestionGenerator,PyIngest
from IPython.display import display, Markdown, Image

load_dotenv()




True

In [ ]:
data = pd.read_excel("/content/Complete_data_Chr14.csv", engine='openpyxl')

In [ ]:
graph = Graph(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"]))




In [ ]:
graph.delete_all()

In [ ]:
sample_list=["NA21134","NA21141","NA20845","NA20852","NA20864","NA20869","NA20849","NA20871","NA20851","NA20876","NA20883","NA20888","NA20890","NA20895","NA20903","NA20908","NA20910","NA21088","NA21090","NA21095","NA21103","NA21108","NA21110",
"NA21115",
"NA21122",
"NA21133",
"NA21127",
"NA21094",
"NA21099",
"NA21102",
"NA21107",
"NA21114",
"NA21119",
"NA21121",
"NA21126",
"NA20856",
"NA20863",
"NA20868",
"NA20870",
"NA20875",
"NA20882",
"NA20887",
"NA20894",
"NA20899",
"NA20902",
"NA20907",
"NA21087",
"NA20846",
"NA20853",
"NA20858",
"NA20872",
"NA20877",
"NA20884",
"NA20889",
"NA20891",
"NA21130",
"NA21135",
"NA21142",
"NA20896",
"NA20904",
"NA20909",
"NA20911",
"NA21089",
"NA20893",
"NA21091",
"NA20898",
"NA20901",
"NA20906",
"NA21086",
"NA21093",
"NA20862",
"NA20867",
"NA20874",
"NA20879",
"NA20881",
"NA20886",
"NA20850",
"NA21137",
"NA21144",
"NA21104",
"NA21109",
"NA21111",
"NA21116",
"NA21123",
"NA21128",
"NA21098",
"NA21101",
"NA21106",
"NA21113",
"NA21118",
"NA21120",
"NA21125",
"NA20847",
"NA20854",
"NA20859",
"NA20861",
"NA20866",
"NA20873",
"NA20878",
"NA20885",
"NA20892",
"NA20897",
"NA20900",
"NA20905",
"NA21092",
"NA21129",
"NA21143",
"NA21097",
"NA21100",
"NA21105",
"NA21112",
"NA21117",
"NA21124"
]

# ***Chromosome KG insert***

In [ ]:
data_com = pd.read_csv("/content/Complete_data_Chr14.csv")


In [ ]:
def insert_data(tx, row, sample_id):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''// Merge the Chromosome node
          MERGE (chr:Chromosome {number: $CHROM})

          // Merge the Variant node (SNP)
          MERGE (SNP:SNP {
            id: $Existing_variation,
            position: $POS,
            ref_allele: $REF,
            alt_allele: $ALT,
            Gene_symbol: $Gene_SYMBOL,
            Gene: $Gene,
            allele_count: toInteger($AC),
            allele_number: toInteger($AN),
            depth: toInteger($DP),
            overall_af: toFloat($AF),
            num_samples: toInteger($NS)
          })
          // Merge the Sample node for the current sample in the sample_list
            MERGE (sample:sample {
              Sample_name: $sample_id
          })
            MERGE (Gene:Gene {
              Gene_Symbol: $Gene_SYMBOL,
              Gene_name: $Gene

          })


          // Create the LOCATED_ON relationship between SNP and Chromosome
          MERGE (SNP)-[:LOCATED_ON]->(chr)

         // Create the LOCATED_ON relationship between SNP and Chromosome
          MERGE (SNP)-[:LOCATED_IN]->(Gene)


          // Merge Population nodes and create HAS_POPULATION_FREQUENCY relationships
          FOREACH (popAF IN [
            {code: 'East Asian', af: $EAS_AF},
            {code: 'European', af: $EUR_AF},
            {code: 'African', af: $AFR_AF},
            {code: 'American', af: $AMR_AF},
            {code: 'South Asian Ancestry', af: $SAS_AF}
          ] |
            MERGE (pop:Superpopulation {name: popAF.code})
            MERGE (SNP)-[r:HAS_POPULATION_FREQUENCY]->(pop)
            SET r.allele_frequency = toFloat(popAF.af)
          )

          // Merge Allele nodes for Reference and Alternate Alleles
          MERGE (refAllele:Allele {nucleotide: $REF})
          MERGE (altAllele:Allele {nucleotide: $ALT})


          // Create the HAS_GENOTYPE relationship between Sample and Variant
          MERGE (sample)-[genotypeRel:HAS_GENOTYPE]->(SNP)
          SET genotypeRel.genotype = $NA20846
          SET genotypeRel.genotype = $NA20847
          SET genotypeRel.genotype = $NA20849
          SET genotypeRel.genotype = $NA20850
          SET genotypeRel.genotype = $NA20851
          SET genotypeRel.genotype = $NA20852
          SET genotypeRel.genotype = $NA20853
          SET genotypeRel.genotype = $NA20854
          SET genotypeRel.genotype = $NA20856
          SET genotypeRel.genotype = $NA20858
          SET genotypeRel.genotype = $NA20861
          SET genotypeRel.genotype = $NA20862
          SET genotypeRel.genotype = $NA20863
          SET genotypeRel.genotype = $NA20864
          SET genotypeRel.genotype = $NA20866
          SET genotypeRel.genotype = $NA20867
          SET genotypeRel.genotype = $NA20868
          SET genotypeRel.genotype = $NA20869
          SET genotypeRel.genotype = $NA20870
          SET genotypeRel.genotype = $NA20872
          SET genotypeRel.genotype = $NA20873
          SET genotypeRel.genotype = $NA20874
          SET genotypeRel.genotype = $NA20875
          SET genotypeRel.genotype = $NA20876
          SET genotypeRel.genotype = $NA20877
          SET genotypeRel.genotype = $NA20878
          SET genotypeRel.genotype = $NA20881
          SET genotypeRel.genotype = $NA20882
          SET genotypeRel.genotype = $NA20883
          SET genotypeRel.genotype = $NA20884
          SET genotypeRel.genotype = $NA20885
          SET genotypeRel.genotype = $NA20886
          SET genotypeRel.genotype = $NA20887
          SET genotypeRel.genotype = $NA20888
          SET genotypeRel.genotype = $NA20889
          SET genotypeRel.genotype = $NA20890
          SET genotypeRel.genotype = $NA20891
          SET genotypeRel.genotype = $NA20892
          SET genotypeRel.genotype = $NA20894
          SET genotypeRel.genotype = $NA20895
          SET genotypeRel.genotype = $NA20896
          SET genotypeRel.genotype = $NA20897
          SET genotypeRel.genotype = $NA20899
          SET genotypeRel.genotype = $NA20900
          SET genotypeRel.genotype = $NA20901
          SET genotypeRel.genotype = $NA20902
          SET genotypeRel.genotype = $NA20903
          SET genotypeRel.genotype = $NA20904
          SET genotypeRel.genotype = $NA20905
          SET genotypeRel.genotype = $NA20906
          SET genotypeRel.genotype = $NA20908
          SET genotypeRel.genotype = $NA20910
          SET genotypeRel.genotype = $NA20911
          SET genotypeRel.genotype = $NA21086
          SET genotypeRel.genotype = $NA21087
          SET genotypeRel.genotype = $NA21088
          SET genotypeRel.genotype = $NA21089
          SET genotypeRel.genotype = $NA21090
          SET genotypeRel.genotype = $NA21091
          SET genotypeRel.genotype = $NA21092
          SET genotypeRel.genotype = $NA21093
          SET genotypeRel.genotype = $NA21094
          SET genotypeRel.genotype = $NA21095
          SET genotypeRel.genotype = $NA21097
          SET genotypeRel.genotype = $NA21098
          SET genotypeRel.genotype = $NA21099
          SET genotypeRel.genotype = $NA21100
          SET genotypeRel.genotype = $NA21101
          SET genotypeRel.genotype = $NA21102
          SET genotypeRel.genotype = $NA21103
          SET genotypeRel.genotype = $NA21104
          SET genotypeRel.genotype = $NA21105
          SET genotypeRel.genotype = $NA21106
          SET genotypeRel.genotype = $NA21107
          SET genotypeRel.genotype = $NA21108
          SET genotypeRel.genotype = $NA21109
          SET genotypeRel.genotype = $NA21110
          SET genotypeRel.genotype = $NA21111



''',{
        **row,  # Include all the other values from the row for SNP, Chromosome, etc.
        "sample_id": sample_id  })

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in data_com.iterrows():
        for sample_id in sample_list:
             session.write_transaction(insert_data, row.to_dict(),sample_id)

<ipython-input-18-955653144e6a>:148: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-18-955653144e6a>:151: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data, row.to_dict(),sample_id)
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('58ab470f.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('35.241.237.34', 7687)))
ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('35.241.237.34', 7687)) (ResolvedIPv4Address(('35.241.237.34', 7687)))


KeyboardInterrupt: 

In [ ]:
data1 = pd.read_csv("/content/Harris country samples.csv")


def insert_data1(tx, row):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''//
          MERGE (sample:sample {
            Sample_name: $Sample_name
          })
           MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (county:county {
            County: "Harris",
            County_FIPS: 48201
          })
         MERGE (Population:Population {
           Population_code: $Population_code,
           Population_name: $Population_name
         })

         MERGE (Superpopulation:Superpopulation {
           Superpopulation_code: $Superpopulation_code,
           Superpopulation_name: $Superpopulation_name
         })

          MERGE (Data_collections:Data_collections {
           Data_collections: $Data_collections
         })

          MERGE (sample)-[:BELONGS_TO]->(Population)
          MERGE (sample)-[:BELONG_TO_SEX]->(Sex)
          MERGE (Population)-[:PART_OF]->(Superpopulation)
          MERGE (sample)-[:IN_DATA_COLLECTION]->(Data_collections)
          MERGE (sample)-[c:COLLECT_FROM]->(county)
          SET c.Years_Range = "2008-2021"


''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in data1.iterrows():
        session.write_transaction(insert_data1, row.to_dict())

<ipython-input-15-10accd944eb8>:46: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-15-10accd944eb8>:48: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data1, row.to_dict())


**NCI DATA TABLE**

In [ ]:
NCI = pd.read_excel("/content/Harris_Texas_Cancer_statistics.xlsx", engine='openpyxl')

def insert_NCI(tx, row):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''//
          MERGE (state:state {
            State: $State
          })

          MERGE (county:county {
            County: $County,
            County_FIPS: $County_FIPS
          })

          MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (Superpopulation:Superpopulation {
                    Superpopulation_code: $Race
         })

         MERGE (Cancer_type:Cancer_Type {
           Cancer_Type: $Cancer_Type
         })

          MERGE (state)-[:HAS_COUNTY]->(county)
          MERGE (Cancer_type)-[:FROM_SEX]->(Sex)
          MERGE (Cancer_type)-[:FROM_RACE]->(Superpopulation)
          MERGE (Superpopulation)-[:IN]->(county)
          MERGE (Cancer_type)-[r:OCCUR_IN]->(county)
          SET r.Years_Range = $Years_range
          SET r.cancer_cases = $number_of_new_cancer_cases_diagnosed
          SET r.Age_Adjusted_Rate_per_100000 = $Age_Adjusted_Rate_per_100000
          SET r.Cancer_Type = $Cancer_Type







''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in NCI.iterrows():
        session.write_transaction(insert_NCI, row.to_dict())

<ipython-input-16-18e0b07c7fac>:49: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-16-18e0b07c7fac>:51: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_NCI, row.to_dict())


***SDOH DATA***

In [ ]:
Education = pd.read_excel("/content/ Harris_Texas_Education.xlsx", engine='openpyxl')


def insert_data_Education(tx, row):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''//

           MERGE (Sex:Sex {
            Sex: $Sex
         })
         MERGE (county:county {
            County: "Harris",
            County_FIPS: 48201
          })
          MERGE (state:state {
            State: $State
          })

         MERGE (Education:Education {
           Education: $Education


         })

         MERGE (statistic:statistic {
           statistic: $Percent
          })


          MERGE (state)-[:HAS_COUNTY]->(county)
          MERGE (statistic)-[r:FOR_SEX]->(Sex)
          SET r.Years_Range = $Years_range
          MERGE (statistic)-[m:FOR_EDUCATION_LEVEL]->(Education)
          SET m.Years_Range = $Years_range
          MERGE (statistic)-[n:RELATES_TO]->(county)
          SET n.Years_Range = $Years_range



''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in Education.iterrows():
        session.write_transaction(insert_data_Education, row.to_dict())

<ipython-input-8-9e9bfc287a3e>:47: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-8-9e9bfc287a3e>:49: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data_Education, row.to_dict())


In [ ]:
Income = pd.read_excel("/content/ Harris_Texas_Income.xlsx", engine='openpyxl')


def insert_data_Income(tx, row):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''//

           MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (county:county {
            County: $County,
            County_FIPS: $County_FIPS
          })
          MERGE (state:state {
            State: $State
          })
          MERGE (Income:Income {
             Income: $Income,
             Dollars: $Dollars
          })
          MERGE (Income)-[n:INCOME_FOR_SEX]->(Sex)
          SET n.Years_Range = $Years_range
          MERGE (Income)-[m:INCOME_FOR_COUNTY]->(county)
          SET m.Years_Range = $Years_range



''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in Income.iterrows():
        session.write_transaction(insert_data_Income, row.to_dict())

<ipython-input-14-be1d65367a20>:36: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-14-be1d65367a20>:38: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data_Income, row.to_dict())


In [8]:
Crowding = pd.read_excel("/content/Harris_Texas_crowding.xlsx", engine='openpyxl')


def insert_data_Crowding(tx, row):

  # Handle potential NaN values by using Python's pandas to check for NaN

    tx.run('''//

           MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (county:county {
            County: $County,
            County_FIPS: $County_FIPS
          })
          MERGE (state:state {
            State: $State
          })
          MERGE (Crowding:Crowding {
             Crowding: $Crowding,
             Percent: $Percent,
             Years_range: $Years_range
          })
          MERGE (Crowding)-[:CROWDING_FOR_SEX]->(Sex)
          MERGE (Crowding)-[:CROWDING_BY_COUNTY]->(county)





''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in Crowding.iterrows():
        session.write_transaction(insert_data_Crowding, row.to_dict())

<ipython-input-8-e8afa6c4de7f>:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-8-e8afa6c4de7f>:39: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data_Crowding, row.to_dict())


In [18]:
Superpopulation = pd.read_excel("/content/Harris_Texas_Population.xlsx", engine='openpyxl')


def insert_data_Superpopulation(tx, row):


    tx.run('''//

           MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (county:county {
            County: $County,
            County_FIPS: $County_FIPS
          })
          MERGE (state:state {
            State: $State
          })

          MERGE (Superpopulation:Superpopulation {
           Superpopulation_code: "",
           Superpopulation_name: $Population
         })

          MERGE (Superpopulation)-[n:Superpopulation_STATISTIC_FOR_SEX]->(Sex)
          SET n.Years_Range = $Years_range
          SET n.Percent= $Percent
          MERGE (Superpopulation)-[m:Superpopulation_STATISTIC_BY_COUNTY]->(county)
          SET m.Years_Range = $Years_range






''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in Superpopulation.iterrows():
        session.write_transaction(insert_data_Superpopulation, row.to_dict())

<ipython-input-18-4a35ceb507e1>:41: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-18-4a35ceb507e1>:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data_Superpopulation, row.to_dict())


In [16]:
Sex = pd.read_excel("/content/Harris_Texas_sex.xlsx", engine='openpyxl')


def insert_data_Sex(tx, row):


    tx.run('''//

           MERGE (Sex:Sex {
            Sex: $Sex
         })
          MERGE (county:county {
            County: $County,
            County_FIPS: $County_FIPS
          })
          MERGE (state:state {
            State: $State
          })

          MERGE (Sex)-[n:Population_cancer_Percent_IN_COUNTY]->(county)
          SET n.Population_cancer_Percent = $Population_cancer_Percent
          SET n.Years_Range = $Years_range






''',row)

with GraphDatabase.driver(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"])) as driver:
    driver.verify_connectivity()

with driver.session() as session:
    for _, row in Sex.iterrows():
        session.write_transaction(insert_data_Sex, row.to_dict())

<ipython-input-16-752d0825972a>:34: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
<ipython-input-16-752d0825972a>:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_data_Sex, row.to_dict())


# ***LLM RAG***

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o",
    openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.5
)

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username="neo4j",
    password=os.environ["NEO4J_PASSWORD"],
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about genetics, SNP data and cancer provide recommendations.
Convert the user's question based on the schema to neo4j cypher query that return the entity's properties and relationships.

Instructions:
Use the provided entity's properties and relationships in the schema:

This knowledge graph is structured around several key entities and the relationships between them. Here is a description of each entity and the relationships it has:

Entities (Nodes):
SNP (Single Nucleotide Polymorphism):

A genetic variation identified by a change in a single nucleotide.
Relationships:
LOCATED_ON → Chromosome: Indicates the chromosome where the SNP is located.
HAS_POPULATION_FREQUENCY → Population: Represents the population frequency data related to the SNP.
LOCATED_IN → Gene: Identifies the gene in which the SNP is located.
HAS_GENOTYPE → sample: Links the SNP to a specific sample and its genotype.
Chromosome:

Represents the chromosome in which a SNP is located.
Relationships:
LOCATED_ON ← SNP: A SNP is located on this chromosome.
Gene:

Represents the gene associated with a particular SNP.
Relationships:
LOCATED_IN ← SNP: The SNP is located within this gene.
Population:

Represents a specific population in which SNP data is available.
Relationships:
HAS_POPULATION_FREQUENCY ← SNP: The population frequency of a specific SNP.
PART_OF → Superpopulation: This population is part of a larger superpopulation.
Superpopulation:

Represents a broader superpopulation that contains several smaller populations.
Relationships:
PART_OF ← Population: Indicates that a population is part of this superpopulation.

sample:

Represents a biological sample used in genomic studies.
Relationships:
HAS_GENOTYPE → SNP: Links the sample to the specific SNP and its genotype.
BELONGS_TO → Population: The sample belongs to a specific population.
COLLECT_FROM → county: Indicates the county where the sample was collected.
BELONG_TO_SEX → Sex: Indicates the sex associated with the sample.

Sex:

Represents the sex (male/female) of a sample.
Relationships:
BELONG_TO_SEX ← sample: The sample is associated with a specific sex.
FROM_SEX → county: Connects the sex to the county in which samples were collected.

county:

Represents a geographic region (county) where samples were collected.
Relationships:
COLLECT_FROM ← sample: Samples are collected from this county.
OCCUR_IN → Cancer_Type: This county is associated with certain cancer types.
FROM_SEX ← Sex: Indicates the sex distribution for the samples collected from this county.
HAS_COUNTY → State: This county is located in a specific state.
State:

Represents a geographic state that contains various counties.
Relationships:
HAS_COUNTY ← county: This state contains the referenced county.
Cancer Type:

Represents different types of cancers observed in a population.
Relationships:
OCCUR_IN ← county: The county is associated with the occurrence of this cancer type.
Data_collections:

Represents different genomic or population data collections.
Relationships:
IN_DATA_COLLECTION → sample: The sample belongs to a specific data collection.



Examples:
For the quastion "What is the best cancer treatment?" use only this cypher:
MATCH (n:Treatment)
RETURN n LIMIT 25;


For the quastion "what is the top 2 letal cancer in harris texas?" use only this cypher:
MATCH (c:Cancer_Type)-[r:OCCUR_IN]->(co:county)<-[:HAS_COUNTY]-(s:state)
WHERE co.county = 'Harris' AND s.State = 'Texas'
RETURN c.Cancer_Type, r.cancer_cases
ORDER BY r.cancer_cases DESC
LIMIT X;

X is by defult is 3 and specify the top 3 cancer types.

For  quastions about SNP use only this cypher:
MATCH (s:sample)-[:HAS_GENOTYPE]->(snp:SNP)
RETURN snp.id, COUNT(s) as sample_count
ORDER BY sample_count DESC
LIMIT X;
and take the 'snp.id' from the Full context as an answer.

For  quastion:"what is the most deadlist genotype?":
MATCH (s:sample)-[hg:HAS_GENOTYPE]->(snp:SNP)
RETURN hg.genotype, COUNT(s) as sample_count
ORDER BY sample_count DESC
LIMIT 1;
and take the 'hg.genotype' and 'sample_count' from the Full context.
answer: "The most deadlist genotype is 'hg.genotype' with 'sample_count' samples".


Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    allow_dangerous_requests=True,
    verbose=True
)

graph_data= cypher_chain.invoke({"query": "what is the most deadlist genotype in texas?"})

print(graph_data)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:sample)-[hg:HAS_GENOTYPE]->(snp:SNP)
RETURN hg.genotype, COUNT(s) as sample_count
ORDER BY sample_count DESC
LIMIT 1;

Full Context:
[{'hg.genotype': '0|0', 'sample_count': 24623}]

> Finished chain.
{'query': 'what is the most deadlist genotype?', 'result': "I don't know the answer."}


In [ ]:
from neo4j import GraphDatabase
import json


# Create a Neo4j driver instance
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def export_nodes(session, output_file):
    # Query to get all nodes
    cypher_query = """
    MATCH (n)
    RETURN id(n) AS node_id, labels(n) AS labels, properties(n) AS properties
    """
    result = session.run(cypher_query)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("Nodes:\n")
        for record in result:
            node_data = {
                'node_id': record['node_id'],
                'labels': record['labels'],
                'properties': record['properties']
            }
            f.write(json.dumps(node_data, ensure_ascii=False) + '\n')

def export_relationships(session, output_file):
    # Query to get all relationships
    cypher_query = """
    MATCH (n)-[r]->(m)
    RETURN id(r) AS rel_id, type(r) AS type, id(n) AS start_node_id, id(m) AS end_node_id, properties(r) AS properties
    """
    result = session.run(cypher_query)

    with open(output_file, 'a', encoding='utf-8') as f:
        f.write("\nRelationships:\n")
        for record in result:
            rel_data = {
                'rel_id': record['rel_id'],
                'type': record['type'],
                'start_node_id': record['start_node_id'],
                'end_node_id': record['end_node_id'],
                'properties': record['properties']
            }
            f.write(json.dumps(rel_data, ensure_ascii=False) + '\n')

def export_knowledge_graph(output_file):
    with driver.session() as session:
        export_nodes(session, output_file)
        export_relationships(session, output_file)

if __name__ == "__main__":
    output_file = 'knowledge_graph_export.txt'
    export_knowledge_graph(output_file)
    print(f"Knowledge graph exported to {output_file}")
    driver.close()


ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('35.241.237.34', 7687)) (ResolvedIPv4Address(('35.241.237.34', 7687)))
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('58ab470f.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('35.241.237.34', 7687)))


Knowledge graph exported to knowledge_graph_export.txt


In [ ]:
#USE FOR CSV FILE
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Step 1: Load the CSV file
csv_file_path = '/content/Complete_data_Chr14.csv'
df = pd.read_csv(csv_file_path, sep=';',on_bad_lines='skip', engine='python')

# Step 2: Concatenate the text data
# Assuming the text you want to process is in a column named 'text'
# Modify 'text' to the appropriate column name in your CSV
column_name = df.columns[0]
text_data = " ".join(df[column_name].astype(str).tolist())

# Create a Document object from the text data
docs = [Document(page_content=text_data)]

# Step 3: Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
documents = documents[:100]



In [ ]:
from langchain.vectorstores import FAISS

def RAG(question: str):

  # Initialize OpenAI Embeddings
  embeddings = OpenAIEmbeddings()
  global documents
  # Generate embeddings for the documents
  texts = [doc.page_content for doc in documents]
  document_embeddings = embeddings.embed_documents(texts)

  # Create Document objects with embeddings
  documents = [Document(page_content=text) for text in texts]

  # Create FAISS index
  faiss_index = FAISS.from_documents(documents, embeddings)

  # Save the index (optional)
  faiss_index.save_local("faiss_index")

  # Perform similarity search
  query = question
  similar_docs = faiss_index.similarity_search(query, k=5)

  search_results = []
  for idx, doc in enumerate(similar_docs):
      result = {
          'document_number': idx + 1,
          'content': doc.page_content,
          'metadata': doc.metadata  # Include metadata if available
      }
      search_results.append(result)


  #print("Search Results Stored in Variable:")
  #for result in search_results:
      #print(f"\nDocument {result['document_number']}:")
      #print(result['content'])
  return search_results

In [ ]:
def full_retriever(question: str):
    #graph_data = graph_retriever(question)
    rag_data = RAG(question)
    vector_data = []
    for el in vector_retriever.invoke(question):
        if isinstance(el.page_content, list):
            content = " ".join(el.page_content)
        else:
            content = el.page_content
        vector_data.append(content)

    final_data = f"""Graph data:
{graph_data}, {rag_data},
vector data:
{"#Document ".join(vector_data)}
    """
    print(final_data)
    return final_data

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input="what is the most deadlist genotype?")

In [ ]:
!pip install sentence-transformers

In [ ]:

import os
from neo4j import GraphDatabase
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from sentence_transformers import SentenceTransformer

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]


# Neo4j connection details
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USER = 'neo4j'
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]

# Initialize Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Initialize OpenAI LLM
llm = OpenAI(temperature=0.0)

# Initialize embedding model
embedding_model = OpenAIEmbeddings()  # Or use SentenceTransformer for local models



# Define prompt template
template = """
You are an expert assistant. Use the context below to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

def get_relevant_data(question):
    with driver.session() as session:
        keywords = extract_keywords(question)
        records = []
        for keyword in keywords:
            cypher_query = """
            MATCH (n)
            WHERE any(prop IN $searchProps WHERE toString(n[prop]) CONTAINS $keyword)
            RETURN n, labels(n) AS labels, n AS properties
            """
            searchProps = ['id', 'name', 'gene_name', 'population_name', 'sample_id']
            result = session.run(cypher_query, searchProps=searchProps, keyword=keyword)
            records.extend(result.data())
    return records

def extract_keywords(text):
    import re
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase and split into words
    words = text.lower().split()
    # Remove common stop words
    stop_words = {'what', 'is', 'the', 'a', 'an', 'of', 'and', 'or', 'in', 'to', 'how', 'do', 'i', 'on', 'for'}
    keywords = [word for word in words if word not in stop_words]
    return keywords


def format_context(data):
    """
    Format the retrieved data into a context string for the LLM.
    """
    context = ""
    for record in data:
        name = record.get('name', '')
        labels = record.get('labels', [])
        properties = record.get('properties', {})
        context += f"Name: {name}\nLabels: {labels}\nProperties: {properties}\n\n"
    return context

def qa_system(question):
    """
    Main function to handle the Q&A process.
    """
    # Step 1: Retrieve relevant data from Neo4j
    data = get_relevant_data(question)
    if not data:
        return "I'm sorry, I couldn't find any information related to your question."

    # Step 2: Format the context
    context = format_context(data)

    # Step 3: Prepare the prompt
    formatted_prompt = prompt.format(context=context, question=question)

    # Step 4: Generate answer using the LLM
    answer = llm(formatted_prompt)

    return answer.strip()

# Example usage
if __name__ == "__main__":
    user_question = input("Please enter your question: ")
    response = qa_system(user_question)
    print("\nAnswer:")
    print(response)



ModuleNotFoundError: No module named 'sentence_transformers'

# ***CHATING WITH THE GRAPH***

In [ ]:
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
# Warning control
import warnings
warnings.filterwarnings("ignore")
llm_type = os.getenv("LLM_TYPE", "GPT_MODEL")
llm = ChatOpenAI(temperature=0, model="gpt-4o")

# ***Vector search***

In [ ]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the Individual, Cancer Type, Genetic Variant, Pathway, SDOH, Geographical Location, Cancer Incidence, Survival Rate and cancer related entities "
        "that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting All the Individual, Cancer Type, Genetic Variant, Pathway, SDOH, Geographical Location, Cancer Incidence, Survival Rate and cancer related entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "Who is Hespanic?"}).names

'Hespanic'

הפונקציה למטה אמורה להחזיר את השם של היישות שיש לה ערך שווה לproprtyvalue

In [ ]:
def get_entity_label_by_property_value(driver, property_value):
    with driver.session() as session:
        query = """
        MATCH (n)
        WHERE any(propName IN keys(n) WHERE n[propName] = $property_value)
        RETURN DISTINCT labels(n) AS node_labels
        """
        result = session.run(query, property_value=property_value)
        labels = set()
        for record in result:
            for label in record['node_labels']:
                labels.add(label)
        return list(labels)

# Example usage:
def get_entity_label_by_property(property_value):


    driver = GraphDatabase.driver(uri=os.environ["NEO4J_URI"],auth=("neo4j", os.environ["NEO4J_PASSWORD"]))



    labels = get_entity_label_by_property_value(driver, property_value)
    return labels

    driver.close()

In [ ]:
from neo4j import  Driver,GraphDatabase
from google.colab import userdata

def initialize_fulltext_index(driver, label):
    with driver.session() as session:
        if isinstance(label, list):
          label = label[0]


        # Step 1: Retrieve all property keys for nodes with the given label
        result = session.run(f"""
        MATCH (n:{label})
        UNWIND keys(n) AS key
        RETURN DISTINCT key
        """)
        properties = [record['key'] for record in result]

        if properties:
            # Step 2: Construct the CREATE INDEX statement
            props_list = ', '.join([f"n.{prop}" for prop in properties])
            index_name = f"{label}_fulltext_index"
            query = f"""
            CREATE FULLTEXT INDEX {index_name} IF NOT EXISTS
            FOR (n:{label})
            ON EACH [{props_list}]
            """
            # Step 3: Execute the index creation query
            session.run(query)
            print(f"Full-text index '{index_name}' created on properties: {props_list}")
        else:
            print(f"No properties found for label '{label}'")


In [ ]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    print(full_text_query)
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """

    result = ""
    entities = entity_chain.invoke({"question": question})

    for entity in entities.names:
        print(entity)
        response = kg.query(
            f"""CALL db.index.fulltext.queryNodes('entity', $query, {{limit:2}})
        YIELD node, score
        CALL {{
          WITH node
          MATCH (names{{id:"{entity}"}})-[r:!MENTIONS]->(neighbor)
          RETURN names.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
          UNION ALL
          WITH node
          MATCH (names{{id:"{entity}"}})<-[r:!MENTIONS]-(neighbor)
          RETURN neighbor.id + ' - ' + type(r) + ' -> ' + names.id AS output
        }}
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        print(response)
        result += "\n".join([el['output'] for el in response])
    return result

# ***Superpopulation graph_retriver***

In [ ]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    print(full_text_query)
    return full_text_query.strip()


# Fulltext index query
def graph_retriever_Superpopulation(question: str) -> str:
    kg = Graph(os.environ["NEO4J_URI"], auth=("neo4j", os.environ["NEO4J_PASSWORD"]))

    """
    Collects the neighborhood of entities mentioned
    in the question
    """

    result = ""
    entities = entity_chain.invoke({"question": question})

    for entity in entities.names:
        print(entity)
        response = kg.query(
            f"""CALL db.index.fulltext.queryNodes('entity', $query, {{limit:2}})
        YIELD node, score
        CALL {{
          WITH node
          MATCH (Superpopulation{{Superpopulation_name:"{entity}"}})-[r:!MENTIONS]->(neighbor)
          RETURN Superpopulation.Superpopulation_name + ' - ' + type(r)+ ' -> ' + neighbor.id AS output
          UNION ALL
          WITH node
          MATCH (Superpopulation{{Superpopulation_name:"{entity}"}})<-[r:!MENTIONS]-(neighbor)
          RETURN neighbor.id + ' - ' + type(r) + ' -> ' + Superpopulation.Superpopulation_name AS output
        }}
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([str(el['output']) for el in response]) # changed this line to stringify the neighbor
    return result

In [ ]:
def get_string_property_keys():
    driver = GraphDatabase.driver(uri=os.environ["NEO4J_URI"],auth=("neo4j", os.environ["NEO4J_PASSWORD"]))
    with driver.session() as session:
        result = session.run("""
            MATCH (n)
            UNWIND keys(n) AS key
            WITH DISTINCT key
            RETURN key
        """)
        property_keys = []
        for record in result:
            key = record["key"]
            # Check if the property values are strings
            prop_result = session.run(f"""
                MATCH (n)
                WHERE n.`{key}` IS NOT NULL
                RETURN n.`{key}` AS value LIMIT 1
            """)
            value = prop_result.single()["value"]
            if isinstance(value, str):
                property_keys.append(key)
        return property_keys

# Get string property keys




def initialize_fulltext_index(entity):
  label_properties = get_string_property_keys()
  neo4j_graph_vector_index = Neo4jVector.from_existing_graph(
      embedding=OpenAIEmbeddings(),
      url=os.getenv("NEO4J_URI"),
      username=os.getenv("NEO4J_USERNAME"),
      password=os.getenv("NEO4J_PASSWORD"),
      node_label=entity[0],
      text_node_properties=label_properties ,
      embedding_node_property="embedding",
)

In [ ]:
superpopulation_details_chat_template_str = """
Your job is to use the provided Race (superpopulation) data to answer
questions about cancer types, sex and samples. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's
not from the context and the entity_info. If you don't know an answer, say you don't know.
{context}
"""
superpopulation_details_chat_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=superpopulation_details_chat_template_str
    )
)
human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="Can you provide details on: {question}?"
    )
)
messages = [superpopulation_details_chat_system_prompt, human_prompt]
qa_prompt = ChatPromptTemplate(
    messages=messages,
    input_variables=["context", "question", "entity_info"]
)
llm = ChatOpenAI(model="gpt-4o-mini")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=neo4j_graph_vector_index.as_retriever(),
    # ['stuff', 'map_reduce', 'refine', 'map_rerank']
    chain_type="stuff",
)
qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt

NameError: name 'neo4j_graph_vector_index' is not defined

In [ ]:
def graphRAG_search(query):
  neo4j_driver = GraphDatabase.driver(uri=os.environ["NEO4J_URI"],auth=("neo4j", os.environ["NEO4J_PASSWORD"]))

  query=query
  superpopulation_details_chat_template_str = """
  Your job is to use the provided Race (superpopulation) data to answer
  questions about cancer types, sex and samples. Use the following context to answer questions.
  Be as detailed as possible, but don't make up any information that's
  not from the context and the entity_info. If you don't know an answer, say you don't know.
  {context}
  """
  superpopulation_details_chat_system_prompt = SystemMessagePromptTemplate(
      prompt=PromptTemplate(
          input_variables=["context"],
          template=superpopulation_details_chat_template_str
      )
  )
  human_prompt = HumanMessagePromptTemplate(
      prompt=PromptTemplate(
          input_variables=["question"],
          template="Can you provide details on: {question}?"
      )
  )
  messages = [superpopulation_details_chat_system_prompt, human_prompt]
  qa_prompt = ChatPromptTemplate(
      messages=messages,
      input_variables=["context", "question", "entity_info"]
  )


  label_properties = get_string_property_keys()


  #query = "What do you know about the Hispanic?"
  search_entity = entity_chain.invoke({"question": query}).names
  search_label= get_entity_label_by_property(search_entity[0])
  neo4j_graph_vector_index = Neo4jVector.from_existing_graph(
        embedding=OpenAIEmbeddings(),
        url=os.getenv("NEO4J_URI"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
        node_label=search_label[0],
        text_node_properties=label_properties ,
        embedding_node_property="embedding",
  )

  lm = ChatOpenAI(model="gpt-4o-mini")

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=neo4j_graph_vector_index.as_retriever(),
      # ['stuff', 'map_reduce', 'refine', 'map_rerank']
      chain_type="stuff",
  )
  qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt



  initialize_fulltext_index(search_label)
  entity_info=graph_retriever_Superpopulation(query)
  #max_entity_info_length = 310
  #entity_info = entity_info[:max_entity_info_length]
  print(entity_info)

  # Step 1: Retrieve context
  retrieved_docs = qa_chain.retriever.get_relevant_documents(query)
  context = "\n".join([doc.page_content for doc in retrieved_docs])

  # Step 2: Combine 'entity_info' into 'context'
  full_context = f"{context}\n the entities and the relationships {entity_info}"
  # Step 3: Invoke the chain
  response = qa_chain.combine_documents_chain.llm_chain.predict(
      context=full_context,
      question=query
  )
  print(response)
  return response


Graph entity extraction

# ***GRAPH_RAG***

In [ ]:
# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

kg.refresh_schema()


In [ ]:
kg.query("""
MATCH (c:Cancer_Type)-[:OCCUR_IN]->(n:county)
RETURN c, n
LIMIT 1
""")

[{'c': {'Cancer_Type': 'Breast'},
  'n': {'County_FIPS': 48201, 'County': 'Harris'}}]

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to
query a graph database.
Instructions:
Use only the provided relationship types and properties in the
schema. Do not use any other relationship types or properties that
are not provided.
Schema:
{schema}
Note:
Do not respond to any questions that might ask anything else than
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher
statements for particular questions:

# what is the most deadliest genotype in texas?
MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)-[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
RETURN hg.genotype, COUNT(hg) as sample_count, hg, s, c, snp
ORDER BY sample_count DESC
LIMIT 1


#What do you know about the Hispanic?
MATCH (sp:Superpopulation)-[r:!MENTIONS]->(n)
WHERE sp.Superpopulation_name = 'Hispanic'
RETURN sp.Superpopulation_name, r, n


The question is:
{question}"""

In [ ]:
cypher_qa_prompt = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Use the question to formulate a full answer.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE
)

qa_prompt = PromptTemplate(
input_variables=["context", "question"], template=cypher_qa_prompt)


llm = ChatOpenAI(model="gpt-4",
    openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.0)

cypherChain = GraphCypherQAChain.from_llm(
    top_k=100,
    llm=llm,
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,
    qa_prompt=qa_prompt
)
def prettyCypherChain(question: str) -> str:
    #g=graphRAG_search(question)
    #context = f"{cypherChain.graph_schema}, {g}"
    #response = cypherChain.run(question)

    response1 = cypherChain.run(question)
    return response1
    print(textwrap.fill(response1, 60))




In [ ]:
prettyCypherChain("what is the most deadliest genotype in texas?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)-[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
RETURN hg.genotype, COUNT(hg) as sample_count, hg, s, c, snp
ORDER BY sample_count DESC
LIMIT 1
Full Context:
[{'hg.genotype': '0|0', 'sample_count': 1, 'hg': ({'Sample_name': 'NA21134'}, 'HAS_GENOTYPE', {'allele_number': 5096, 'allele_count': 231, 'depth': 9775, 'Gene': '-', 'alt_allele': 'T', 'id': 'rs1454849680', 'position': 16038675, 'overall_af': 0.05, 'Gene_symbol': '-', 'num_samples': 2548, 'ref_allele': 'C'}), 's': {'Sample_name': 'NA21134'}, 'c': {'County_FIPS': 48201, 'id': 'Harris'}, 'snp': {'allele_number': 5096, 'allele_count': 231, 'depth': 9775, 'Gene': '-', 'alt_allele': 'T', 'id': 'rs1454849680', 'position': 16038675, 'overall_af': 0.05, 'Gene_symbol': '-', 'num_samples': 2548, 'ref_allele': 'C'}}]

> Finished chain.


"The sample named NA21134 has a genotype of 0|0. This sample was tested for the SNP with the ID rs1454849680. The SNP is located at position 16038675 and has a reference allele of 'C' and an alternate allele of 'T'. The overall allele frequency is 0.05. The total number of samples tested for this SNP is 2548, and the allele count is 231 out of an allele number of 5096. The depth of the sequencing data is 9775. The gene associated with this SNP is not specified. The sample comes from Harris county with the FIPS code 48201."

Respon 2

In [ ]:
CYPHER_GENERATION_TEMPLATE2 = """Task:Generate Cypher statement to
query a graph database.
Instructions:
Use only the provided relationship types and properties in the
schema. Do not use any other relationship types or properties that
are not provided.
Schema:
{schema}
Note:
Do not respond to any questions that might ask anything else than
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher
statements for particular questions:




# what is the most deadliest genotype in texas?
MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
WITH hg.genotype AS genotype, COUNT(*) AS genotype_count,s.Sample_name AS Sample_name

ORDER BY genotype_count DESC
LIMIT 1
RETURN Sample_name +' '+ 'with genotype' +' '+ genotype + ' ' + genotype_count + ' sample_count 'AS output

UNION ALL

MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
WITH hg.genotype AS genotype, COUNT(*) AS genotype_count,s.Sample_name AS Sample_name1

ORDER BY genotype_count DESC
LIMIT 1
MATCH (s:sample)-[r:!MENTIONS]->(neighbor)
WHERE s.Sample_name = Sample_name1
RETURN s.Sample_name + ' - ' + type(r)+ ' -> ' + neighbor.id AS output


UNION ALL

MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
WITH hg.genotype AS genotype, COUNT(*) AS genotype_count,s.Sample_name AS Sample_name1

ORDER BY genotype_count DESC
LIMIT 1
MATCH (s:sample)<-[r:!MENTIONS]-(neighbor)
WHERE s.Sample_name = Sample_name1
RETURN neighbor.id+ ' - ' + type(r)+ ' -> ' + s.Sample_name AS output





#What do you know about the Hispanic?
MATCH (sp:Superpopulation)-[r:!MENTIONS]->(n)
WHERE sp.Superpopulation_name = 'Hispanic'
RETURN sp.Superpopulation_name, r, n


The question is:
{question}"""

In [ ]:
cypher_qa_prompt2 = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Use the question to formulate a full answer.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE2
)

qa_prompt = PromptTemplate(
input_variables=["context", "question"], template=cypher_qa_prompt2)


llm = ChatOpenAI(model="gpt-4",
    openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.0)

cypherChain2 = GraphCypherQAChain.from_llm(
    top_k=100,
    llm=llm,
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,
    qa_prompt=qa_prompt
)
def prettyCypherChain2(question: str) -> str:
    #g=graphRAG_search(question)
    #context = f"{cypherChain.graph_schema}, {g}"
    #response = cypherChain.run(question)

    response2 = cypherChain2.run(question)
    return response2
    print(textwrap.fill(response1, 60))




In [ ]:
prettyCypherChain2("what is the most deadliest genotype in texas?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
WITH hg.genotype AS genotype, COUNT(*) AS genotype_count,s.Sample_name AS Sample_name
     
ORDER BY genotype_count DESC
LIMIT 1 
RETURN Sample_name +' '+ 'with genotype' +' '+ genotype + ' ' + genotype_count + ' sample_count 'AS output

UNION ALL

MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
WITH hg.genotype AS genotype, COUNT(*) AS genotype_count,s.Sample_name AS Sample_name1
     
ORDER BY genotype_count DESC
LIMIT 1 
MATCH (s:sample)-[r:!MENTIONS]->(neighbor)
WHERE s.Sample_name = Sample_name1
RETURN s.Sample_name + ' - ' + type(r)+ ' -> ' + neighbor.id AS output


UNION ALL

MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)
      -[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'

Full Context:
[{'output': 'NA21134 with genotype 0|0 231 sample_count '}, {'output': None}, {'output': None}, {'output': None}, {'output': 'NA21134 - COLLECT_FROM -> Harris'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1454849680'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1160632320'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1473857766'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1228971855'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1176286927'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1427290039'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1247236590'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1302756061'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1381118581'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1236756715'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1345872846'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1332670268'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1293182703'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1351481019'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs936990738'}, {'output': 'NA2

'The sample NA21134 has a genotype of 0|0 with a sample count of 231. It was collected from Harris. The sample NA21134 has the following genotypes: rs1454849680, rs1160632320, rs1473857766, rs1228971855, rs1176286927, rs1427290039, rs1247236590, rs1302756061, rs1381118581, rs1236756715, rs1345872846, rs1332670268, rs1293182703, rs1351481019, rs936990738, rs916816458, rs2139620105, rs1271109451, rs2089204274, rs1004947043, rs1168556733, rs1028948669, rs949576520, rs1311846483, rs2089204865, rs890392704, rs1418238236, rs1000434954, rs1175722323, rs1201458425, rs940876481, rs1314848291, rs1050465694, rs944691826, rs894243612, rs1345024770, rs1367181594, rs2139620330, rs973616940, rs920734544, rs911976120, rs944745721, rs2139620363, rs924532256, rs935890754, rs894326071, rs1045545862, rs1015246115, rs986256488, rs911987237, rs966059368, rs1200736942, rs936000753, rs1346706359, rs1276917611, rs915827838, rs948593697, rs1240659911, rs1213365466, rs1007686014, rs1458033728, rs2089208175, rs98

# ***Final integration***

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# Define the prompt template
prompt_template = """
You are an assistant that combines two pieces of information into a singal answer.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Use the question to formulate a full answer.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.

Information 1:
{output1}

Information 2:
{output2}

Please provide a combined summary of the above information.
"""

prompt = PromptTemplate(
    input_variables=["output1", "output2"],
    template=prompt_template
)

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.7
)

# Create an LLMChain with the prompt and model
chain = LLMChain(llm=llm, prompt=prompt)

# Define the outputs you want to combine
output1 = prettyCypherChain("what is the most deadliest genotype in texas?")
output2 = prettyCypherChain2("what is the most deadliest genotype in texas?")

# Run the chain with the inputs
combined_summary = chain.run(output1=output1, output2=output2)

# Print the result
print(textwrap.fill(combined_summary, 60))





> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (st:state)-[:HAS_COUNTY]->(c:county)<-[:COLLECT_FROM]-(s:sample)-[hg:HAS_GENOTYPE]->(snp:SNP)
WHERE st.State = 'Texas'
RETURN hg.genotype, COUNT(hg) as sample_count, hg, s, c, snp
ORDER BY sample_count DESC
LIMIT 1
Full Context:
[{'hg.genotype': '0|0', 'sample_count': 1, 'hg': ({'Sample_name': 'NA21134'}, 'HAS_GENOTYPE', {'allele_number': 5096, 'allele_count': 231, 'depth': 9775, 'Gene': '-', 'alt_allele': 'T', 'id': 'rs1454849680', 'position': 16038675, 'overall_af': 0.05, 'Gene_symbol': '-', 'num_samples': 2548, 'ref_allele': 'C'}), 's': {'Sample_name': 'NA21134'}, 'c': {'County_FIPS': 48201, 'id': 'Harris'}, 'snp': {'allele_number': 5096, 'allele_count': 231, 'depth': 9775, 'Gene': '-', 'alt_allele': 'T', 'id': 'rs1454849680', 'position': 16038675, 'overall_af': 0.05, 'Gene_symbol': '-', 'num_samples': 2548, 'ref_allele': 'C'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH

Full Context:
[{'output': 'NA21134 with genotype 0|0 231 sample_count '}, {'output': None}, {'output': None}, {'output': None}, {'output': 'NA21134 - COLLECT_FROM -> Harris'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1454849680'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1160632320'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1473857766'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1228971855'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1176286927'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1427290039'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1247236590'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1302756061'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1381118581'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1236756715'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1345872846'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1332670268'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1293182703'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs1351481019'}, {'output': 'NA21134 - HAS_GENOTYPE -> rs936990738'}, {'output': 'NA2